# Semantic Kernel Chat Completion Agent

Chat Completion is fundamentally a protocol for a chat-based interaction with an AI model where the chat-history maintained and presented to the model with each request. Semantic Kernel AI services offer a unified framework for integrating the chat-completion capabilities of various AI models.

A chat completion agent can leverage any of these AI services to generate responses, whether directed to a user or another agent.


## Creating the required Azure resources

You can run the powershell script `create-azure-ai-resources.ps1` to create the following resources:
* Azure AI Services with Hub and Project
* GPT-4o model

## Installing the necessary packages

Import Semantic Kernel SDK from pypi.org

In [1]:
%pip install -U semantic-kernel --quiet

Note: you may need to restart the kernel to use updated packages.


Check the current version of Semantic Kernel.

In [2]:
from semantic_kernel import __version__

__version__

'1.23.1'

In [3]:
from dotenv import load_dotenv
import os

if os.path.exists(".env"):
    load_dotenv(override=True)

## Creating a chat completion service

>Note: The AzureChatCompletion service also supports Microsoft Entra authentication. If you don't provide an API key, the service will attempt to authenticate using the Entra token.

In [18]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    service_id="service1"
)

Let's try chat with LLM model.

In [19]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.contents.chat_history import ChatHistory

execution_settings = AzureChatPromptExecutionSettings()

system_message = """
You are a helpful assistant that can answer questions and provide information.
"""

chat_history = ChatHistory(system_message=system_message)

chat_history.add_user_message("Hello, how are you?")

response = await chat_completion.get_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings,
)

print(response)

Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


## Creating a Chat Completion Agent

A chat completion agent is fundamentally based on an AI services. As such, creating an chat completion agent starts with creating a Kernel instance that contains one or more chat-completion services and then instantiating the agent with a reference to that Kernel instance.

In [21]:
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings

# Define the Kernel
kernel = Kernel()

# Add the AzureChatCompletion AI Service to the Kernel
kernel.add_service(chat_completion)

settings = AzureChatPromptExecutionSettings(service_id="service1")

# Create the agent
agent = ChatCompletionAgent(
    kernel=kernel, 
    name="chat-agent", 
    instructions="You are a helpful agent.",
    arguments=KernelArguments(settings=settings)
)

## Conversing with ChatCompletionAgent

There are multiple ways to converse with a ChatCompletionAgent.

The easiest is to call and await get_response:

In [22]:
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents import AuthorRole

# Define the chat history
history = ChatHistory()

# Add the user message
history.add_message(ChatMessageContent(role=AuthorRole.USER, content="Hello, how are you?"))

# Generate the agent response
response = await agent.get_response(history)

print(f"{response.content}")

Hello! I'm just a virtual assistant, but I'm here and ready to help you. How can I assist you today?


## Creating a Semantic Kernel Plugin

Let's create a plugin that interacts with Github API for repositories and users.

This plugin example was provided by Microsoft at this link: https://github.com/microsoft/semantic-kernel/blob/main/python/samples/learn_resources/plugins/GithubPlugin/github.py

In [10]:
# Copyright (c) Microsoft. All rights reserved.


import httpx
from pydantic import BaseModel, Field

from semantic_kernel.functions.kernel_function_decorator import kernel_function

# region GitHub Models


class Repo(BaseModel):
    id: int = Field(..., alias="id")
    name: str = Field(..., alias="full_name")
    description: str | None = Field(default=None, alias="description")
    url: str = Field(..., alias="html_url")


class User(BaseModel):
    id: int = Field(..., alias="id")
    login: str = Field(..., alias="login")
    name: str | None = Field(default=None, alias="name")
    company: str | None = Field(default=None, alias="company")
    url: str = Field(..., alias="html_url")


class Label(BaseModel):
    id: int = Field(..., alias="id")
    name: str = Field(..., alias="name")
    description: str | None = Field(default=None, alias="description")


class Issue(BaseModel):
    id: int = Field(..., alias="id")
    number: int = Field(..., alias="number")
    url: str = Field(..., alias="html_url")
    title: str = Field(..., alias="title")
    state: str = Field(..., alias="state")
    labels: list[Label] = Field(..., alias="labels")
    when_created: str | None = Field(default=None, alias="created_at")
    when_closed: str | None = Field(default=None, alias="closed_at")


class IssueDetail(Issue):
    body: str | None = Field(default=None, alias="body")


# endregion


class GitHubSettings(BaseModel):
    base_url: str = "https://api.github.com"
    token: str


class GitHubPlugin:
    def __init__(self, settings: GitHubSettings):
        self.settings = settings

    @kernel_function
    async def get_user_profile(self) -> "User":
        async with self.create_client() as client:
            response = await self.make_request(client, "/user")
            return User(**response)

    @kernel_function
    async def get_repository(self, organization: str, repo: str) -> "Repo":
        async with self.create_client() as client:
            response = await self.make_request(client, f"/repos/{organization}/{repo}")
            return Repo(**response)

    @kernel_function
    async def get_issues(
        self,
        organization: str,
        repo: str,
        max_results: int | None = None,
        state: str = "",
        label: str = "",
        assignee: str = "",
    ) -> list["Issue"]:
        async with self.create_client() as client:
            path = f"/repos/{organization}/{repo}/issues?"
            path = self.build_query(path, "state", state)
            path = self.build_query(path, "assignee", assignee)
            path = self.build_query(path, "labels", label)
            path = self.build_query(path, "per_page", str(max_results) if max_results else "")
            response = await self.make_request(client, path)
            return [Issue(**issue) for issue in response]

    @kernel_function
    async def get_issue_detail(self, organization: str, repo: str, issue_id: int) -> "IssueDetail":
        async with self.create_client() as client:
            path = f"/repos/{organization}/{repo}/issues/{issue_id}"
            response = await self.make_request(client, path)
            return IssueDetail(**response)

    def create_client(self) -> httpx.AsyncClient:
        headers = {
            "User-Agent": "request",
            "Accept": "application/vnd.github+json",
            "Authorization": f"Bearer {self.settings.token}",
            "X-GitHub-Api-Version": "2022-11-28",
        }
        return httpx.AsyncClient(base_url=self.settings.base_url, headers=headers, timeout=5)

    @staticmethod
    def build_query(path: str, key: str, value: str) -> str:
        if value:
            return f"{path}{key}={value}&"
        return path

    @staticmethod
    async def make_request(client: httpx.AsyncClient, path: str) -> dict:
        print(f"REQUEST: {path}\n")
        response = await client.get(path)
        response.raise_for_status()
        return response.json()

## Agent Definition

Finally we are ready to instantiate a ChatCompletionAgent with its Instructions, associated Kernel, and the default Arguments and Execution Settings. In this case, we desire to have the any plugin functions automatically executed.

>You will need to define settings for either OpenAI or Azure OpenAI and also for GitHub.

>Note: For information on GitHub Personal Access Tokens, see: [Managing your personal access tokens](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens).

In [11]:
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.kernel import Kernel

# Import GitHubPlugin and GitHubSettings directly from the notebook
from __main__ import GitHubPlugin, GitHubSettings

kernel = Kernel()
# Add the AzureChatCompletion AI Service to the Kernel
service_id = "agent"
kernel.add_service(AzureChatCompletion(service_id=service_id))

settings = kernel.get_prompt_execution_settings_from_service_id(service_id=service_id)
# Configure the function choice behavior to auto invoke kernel functions
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Set your GitHub Personal Access Token (PAT) value here
gh_settings = GitHubSettings(token=os.getenv("GITHUB_PAT"))

kernel.add_plugin(plugin=GitHubPlugin(gh_settings), plugin_name="GithubPlugin")

# Create the agent
agent = ChatCompletionAgent(
    kernel=kernel,
    name="SampleAssistantAgent",
    instructions=f"""
        You are an agent designed to query and retrieve information from a single GitHub repository in a read-only manner.
        You are also able to access the profile of the active user.
        Use the current date and time to provide up-to-date details or time-sensitive responses.
        The repository you are querying is a public repository with the following name: HoussemDellai/ai-course
        The current date and time is: {{$now}}. 
        """,
    arguments=KernelArguments(settings=settings),
)

Let's get the username of the current user.

In [12]:
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from datetime import datetime

history = ChatHistory()

user_input = "What is my username?"

history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

arguments = KernelArguments(now=datetime.now().strftime("%Y-%m-%d %H:%M"))

async for response in agent.invoke(history=history, arguments=arguments):
    print(f"{response.content}")

REQUEST: /user

Your username is **HoussemDellai**.


Describe the repo.

In [13]:
user_input = "Describe the repo."

history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

arguments = KernelArguments(now=datetime.now().strftime("%Y-%m-%d %H:%M"))

async for response in agent.invoke(history=history, arguments=arguments):
    print(f"{response.content}")

REQUEST: /repos/HoussemDellai/ai-course

The repository **HoussemDellai/ai-course** is focused on "Learning Azure AI with APIM, Semantic Kernel and LangChain." You can visit the repository for more details [here](https://github.com/HoussemDellai/ai-course).


Describe the newest issue created in the repo.

In [14]:
user_input = "Describe the newest issue created in the repo."

history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

arguments = KernelArguments(now=datetime.now().strftime("%Y-%m-%d %H:%M"))

async for response in agent.invoke(history=history, arguments=arguments):
    print(f"{response.content}")

REQUEST: /repos/HoussemDellai/ai-course/issues?state=open&per_page=1&

The newest issue created in the repository "HoussemDellai/ai-course" is titled "[Creating issue just for testing in a chat agent](https://github.com/HoussemDellai/ai-course/issues/1)." It is currently open and was created on March 7, 2025. The issue is labeled as a "bug" with the description "Something isn't working."


List the top 10 issues closed within the last week.

In [15]:
user_input = "List the top 10 issues closed within the last week."

history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

arguments = KernelArguments(now=datetime.now().strftime("%Y-%m-%d %H:%M"))

async for response in agent.invoke(history=history, arguments=arguments):
    print(f"{response.content}")

REQUEST: /repos/HoussemDellai/ai-course/issues?state=closed&per_page=10&

Here is the top issue that was closed within the last week in the repository "HoussemDellai/ai-course":

1. **Title**: [Adding langchain samples](https://github.com/HoussemDellai/ai-course/issues/2)
   - **State**: Closed
   - **Labels**: Bug, Documentation, Question
   - **Created On**: March 7, 2025
   - **Closed On**: March 7, 2025

This is the only closed issue within the past week.


How were these issues labeled?

In [16]:
user_input = "How were these issues labeled?"

history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

arguments = KernelArguments(now=datetime.now().strftime("%Y-%m-%d %H:%M"))

async for response in agent.invoke(history=history, arguments=arguments):
    print(f"{response.content}")

The issue that was closed within the last week, titled "Adding langchain samples," was labeled with the following tags:

1. **bug**: Something isn't working
2. **documentation**: Improvements or additions to documentation
3. **question**: Further information is requested


List the 5 most recently opened issues with the "Bug" label.

In [17]:
user_input = "List the 5 most recently opened issues with the 'Bug' label."

history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

arguments = KernelArguments(now=datetime.now().strftime("%Y-%m-%d %H:%M"))

async for response in agent.invoke(history=history, arguments=arguments):
    print(f"{response.content}")

REQUEST: /repos/HoussemDellai/ai-course/issues?state=open&labels=bug&per_page=5&

Here is the most recently opened issue with the 'Bug' label:

1. **Issue #1**: [Creating issue just for testing in a chat agent](https://github.com/HoussemDellai/ai-course/issues/1)
   - **State**: Open
   - **Labels**: Bug
   - **Created On**: March 7, 2025
   - **Description**: This issue was likely created for testing purposes.


## More resources

https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/examples/example-chat-agent?pivots=programming-language-python